In [14]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import os, json, math, time

from tqdm.notebook import tqdm_notebook

In [15]:
#Load in API credentials 

import json
with open ('/Users/bandj/.secret/yelp_api.json') as f: 
    login = json.load(f)

login.keys()

dict_keys(['client-id', 'api-key'])

In [18]:
from yelpapi import YelpAPI
yelp = YelpAPI(login['api-key'], timeout_s = 5.0)
yelp

In [20]:
location = 'Seattle, WA 98122'
term = 'pizza'

In [21]:
location.split(',')[0]

'Seattle'

In [27]:
#Specify folder for saving the data
folder = 'Data/'

os.makedirs(folder, exist_ok=True)
json_file = folder+f"{location.split(',')[0]}-{term}.json"

In [28]:
json_file

'Data/Seattle-pizza.json'

In [34]:
#Check if JSON file exists, create it if it doesnt
file_exists = os.path.isfile(json_file)

#if file doesn't exist
if file_exists == False:
    folder = os.path.dirname(json_file)
    
    if len(folder) > 0:
        os.makedirs(folder, exist_ok = True)
        
#Inform user and save in empty list

    print(f'[i]{json_file} not found. Saving empty list to file.')
    
    with open (json_file, 'w') as f:
        json.dump([],f)
        
else:
    print(f'[i] {json_file} already exists.')

[i] Data/Seattle-pizza.json already exists.


In [35]:
#Make first API call to get the first page of data
results = yelp.search_query(term=term, location = location)
type(results)

dict

In [36]:
len(results)

3

In [37]:
results.keys()

dict_keys(['businesses', 'total', 'region'])

In [39]:
results['total']

841

In [40]:
results['region']

{'center': {'longitude': -122.29568481445312, 'latitude': 47.61309534151991}}

In [41]:
results['businesses']

[{'id': 'FVbwpNA1uZEGiM02N4XtUg',
  'alias': 'blotto-seattle',
  'name': 'Blotto',
  'image_url': 'https://s3-media3.fl.yelpcdn.com/bphoto/Q4Y1CwKmapk89MYpoxCu7Q/o.jpg',
  'is_closed': False,
  'url': 'https://www.yelp.com/biz/blotto-seattle?adjust_creative=M_n_c0bs2T-9fpMzepv-5g&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=M_n_c0bs2T-9fpMzepv-5g',
  'review_count': 47,
  'categories': [{'alias': 'pizza', 'title': 'Pizza'}],
  'rating': 4.5,
  'coordinates': {'latitude': 47.61849, 'longitude': -122.31664},
  'transactions': [],
  'location': {'address1': '1830 12th Ave',
   'address2': '',
   'address3': None,
   'city': 'Seattle',
   'zip_code': '98122',
   'country': 'US',
   'state': 'WA',
   'display_address': ['1830 12th Ave', 'Seattle, WA 98122']},
  'phone': '+12064031809',
  'display_phone': '(206) 403-1809',
  'distance': 1686.8621759079958},
 {'id': 'EwrAoy7XbX8nfemZdDvupA',
  'alias': 'cornelly-seattle',
  'name': 'Cornelly',
  'image_url': 'https://

In [42]:
#how many results total?
pd.DataFrame(results['businesses'])

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,location,phone,display_phone,distance,price
0,FVbwpNA1uZEGiM02N4XtUg,blotto-seattle,Blotto,https://s3-media3.fl.yelpcdn.com/bphoto/Q4Y1Cw...,False,https://www.yelp.com/biz/blotto-seattle?adjust...,47,"[{'alias': 'pizza', 'title': 'Pizza'}]",4.5,"{'latitude': 47.61849, 'longitude': -122.31664}",[],"{'address1': '1830 12th Ave', 'address2': '', ...",+12064031809,(206) 403-1809,1686.862176,NaN
1,EwrAoy7XbX8nfemZdDvupA,cornelly-seattle,Cornelly,https://s3-media4.fl.yelpcdn.com/bphoto/HlYYBT...,False,https://www.yelp.com/biz/cornelly-seattle?adju...,110,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",4.5,"{'latitude': 47.624577, 'longitude': -122.325577}","[pickup, delivery]","{'address1': '601 Summit Ave E', 'address2': '...",,,2578.701393,$$
2,fxyHWmfzcdjImgQ_UYHoTw,bar-cotto-seattle,Bar Cotto,https://s3-media3.fl.yelpcdn.com/bphoto/07DoPF...,False,https://www.yelp.com/biz/bar-cotto-seattle?adj...,355,"[{'alias': 'bars', 'title': 'Bars'}, {'alias':...",4.5,"{'latitude': 47.6146934, 'longitude': -122.312...","[pickup, delivery]","{'address1': '1546 15th Ave', 'address2': '', ...",+12068388081,(206) 838-8081,1292.526796,$$
3,Wi6LFkjIausYj277ru6pqg,humble-pie-seattle,Humble Pie,https://s3-media4.fl.yelpcdn.com/bphoto/A08GOZ...,False,https://www.yelp.com/biz/humble-pie-seattle?ad...,369,"[{'alias': 'pizza', 'title': 'Pizza'}]",4.0,"{'latitude': 47.5976491915013, 'longitude': -1...",[delivery],"{'address1': '525 Rainier Ave S', 'address2': ...",+12063295133,(206) 329-5133,2166.834011,$$
4,t-Z9bvmlgUyDtomGmttrUQ,guerilla-pizza-kitchen-seattle,Guerilla Pizza Kitchen,https://s3-media4.fl.yelpcdn.com/bphoto/uMrdEj...,False,https://www.yelp.com/biz/guerilla-pizza-kitche...,1,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",5.0,"{'latitude': 47.608127, 'longitude': -122.302435}",[],"{'address1': '2300 E Cherry St', 'address2': '...",,,749.173113,NaN
5,M9xzvwgK58T0w7wvXedvuQ,hot-mamas-pizza-seattle,Hot Mama's Pizza,https://s3-media4.fl.yelpcdn.com/bphoto/LEL1qj...,False,https://www.yelp.com/biz/hot-mamas-pizza-seatt...,969,"[{'alias': 'pizza', 'title': 'Pizza'}]",4.0,"{'latitude': 47.615379179632, 'longitude': -12...","[pickup, delivery]","{'address1': '700 E Pine St', 'address2': '', ...",+12063226444,(206) 322-6444,2080.359405,$
6,ugTsEtjvwRhteac_6JcuPw,italian-family-pizza-seattle,Italian Family Pizza,https://s3-media1.fl.yelpcdn.com/bphoto/-aZrUu...,False,https://www.yelp.com/biz/italian-family-pizza-...,1009,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",4.0,"{'latitude': 47.60937, 'longitude': -122.32546}","[pickup, delivery]","{'address1': '1028 Madison St', 'address2': No...",+12065380040,(206) 538-0040,2271.307105,$$
7,IrohtoYjnAR_vc6w6CRCxA,moto-seattle,Moto,https://s3-media2.fl.yelpcdn.com/bphoto/RA8N1B...,False,https://www.yelp.com/biz/moto-seattle?adjust_c...,164,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",4.5,"{'latitude': 47.56207, 'longitude': -122.38509}",[delivery],"{'address1': '4526 42nd Ave SW', 'address2': '...",+12064208880,(206) 420-8880,8784.075722,$$
8,-FOAQv22SXtSBs7nptI3UA,serious-pie-downtown-seattle-2,Serious Pie Downtown,https://s3-media1.fl.yelpcdn.com/bphoto/Yge-Xa...,False,https://www.yelp.com/biz/serious-pie-downtown-...,4512,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",4.0,"{'latitude': 47.61304626911818, 'longitude': -...","[pickup, delivery]","{'address1': '2001 4th Ave', 'address2': None,...",+12068387388,(206) 838-7388,3348.309276,$$
9,GJdmwqqkC0nAQfX1qZM3ew,olympia-pizza-and-spaghetti-house-iii-seattle,Olympia Pizza and Spaghetti House III,https://s3-media1.fl.yelpcdn.com/bphoto/KvTgMA...,False,https://www.yelp.com/biz/olympia-pizza-and-spa...,299,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",4.0,"{'latitude': 47.62383, 'longitude': -122.312445}",[delivery],"{'address1': '516 15th Ave E', 'address2': '',...",+12063294500,(206) 329-4500,1732.874383,$$


In [43]:
#Where is the actual data we want to save?
#How many did we get the details for?
results_per_page = len(results['businesses'])
results_per_page

20

In [44]:
#Calculate how many pages of results needed to cover the total_results
n_pages = math.ceil((results['total'])/results_per_page)
n_pages

43

In [47]:
for i in tqdm_notebook(range(1,n_pages+1)):
    #the block of code we need to TRY to run
    try:
        time.sleep(.2)
        
        #Read in results in progress file and check the length
        with open (json_file, 'r') as f:
            previous_results = json.load(f)
            
        #Save number of results for to use as offset
        n_results = len(previous_results)
        
        results = yelp.search_query(location=location, term=term,
                                   offset = n_results+1)
        
        #append new results and save to file
        previous_results.extend(results['businesses'])
        
        with open (json_file, 'w') as f:
            json.dump(previous_results, f)
        
    #What to do if we get an error / exception:
    except Exception as e:
        print(' [!] ERROR', e)

  0%|          | 0/43 [00:00<?, ?it/s]

In [48]:
#Open the final json file with pandas
df = pd.read_json(json_file)
df.head()

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
0,EwrAoy7XbX8nfemZdDvupA,cornelly-seattle,Cornelly,https://s3-media4.fl.yelpcdn.com/bphoto/HlYYBT...,False,https://www.yelp.com/biz/cornelly-seattle?adju...,110,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",4.5,"{'latitude': 47.624577, 'longitude': -122.325577}","[delivery, pickup]",$$,"{'address1': '601 Summit Ave E', 'address2': '...",,,2578.701393
1,fxyHWmfzcdjImgQ_UYHoTw,bar-cotto-seattle,Bar Cotto,https://s3-media3.fl.yelpcdn.com/bphoto/07DoPF...,False,https://www.yelp.com/biz/bar-cotto-seattle?adj...,355,"[{'alias': 'bars', 'title': 'Bars'}, {'alias':...",4.5,"{'latitude': 47.6146934, 'longitude': -122.312...","[delivery, pickup]",$$,"{'address1': '1546 15th Ave', 'address2': '', ...",+12068388081,(206) 838-8081,1292.526796
2,Wi6LFkjIausYj277ru6pqg,humble-pie-seattle,Humble Pie,https://s3-media4.fl.yelpcdn.com/bphoto/A08GOZ...,False,https://www.yelp.com/biz/humble-pie-seattle?ad...,369,"[{'alias': 'pizza', 'title': 'Pizza'}]",4.0,"{'latitude': 47.5976491915013, 'longitude': -1...",[delivery],$$,"{'address1': '525 Rainier Ave S', 'address2': ...",+12063295133,(206) 329-5133,2166.834011
3,t-Z9bvmlgUyDtomGmttrUQ,guerilla-pizza-kitchen-seattle,Guerilla Pizza Kitchen,https://s3-media4.fl.yelpcdn.com/bphoto/uMrdEj...,False,https://www.yelp.com/biz/guerilla-pizza-kitche...,1,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",5.0,"{'latitude': 47.608127, 'longitude': -122.302435}",[],NaN,"{'address1': '2300 E Cherry St', 'address2': '...",,,749.173113
4,M9xzvwgK58T0w7wvXedvuQ,hot-mamas-pizza-seattle,Hot Mama's Pizza,https://s3-media4.fl.yelpcdn.com/bphoto/LEL1qj...,False,https://www.yelp.com/biz/hot-mamas-pizza-seatt...,969,"[{'alias': 'pizza', 'title': 'Pizza'}]",4.0,"{'latitude': 47.615379179632, 'longitude': -12...","[delivery, pickup]",$,"{'address1': '700 E Pine St', 'address2': '', ...",+12063226444,(206) 322-6444,2080.359405


In [49]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 840 entries, 0 to 839
Data columns (total 16 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             840 non-null    object 
 1   alias          840 non-null    object 
 2   name           840 non-null    object 
 3   image_url      840 non-null    object 
 4   is_closed      840 non-null    bool   
 5   url            840 non-null    object 
 6   review_count   840 non-null    int64  
 7   categories     840 non-null    object 
 8   rating         840 non-null    float64
 9   coordinates    840 non-null    object 
 10  transactions   840 non-null    object 
 11  price          740 non-null    object 
 12  location       840 non-null    object 
 13  phone          840 non-null    object 
 14  display_phone  840 non-null    object 
 15  distance       840 non-null    float64
dtypes: bool(1), float64(2), int64(1), object(12)
memory usage: 99.4+ KB


In [50]:
#convert the filename to a .csv.gz
csv_file = json_file.replace('.json', 'csv.gz')
csv_file

'Data/Seattle-pizzacsv.gz'

In [52]:
#Save it as a compressed csv to save space
df.to_csv(csv_file, compression='gzip', index=False)